In [23]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

shape = gpd.read_file('data/calenviroscreen40_shp/CES4 Final Shapefile.shp')
shape = shape.replace([-999, -1998], np.nan)

In [24]:
shape.columns

Index(['Tract', 'ZIP', 'County', 'ApproxLoc', 'TotPop19', 'CIscore',
       'CIscoreP', 'Ozone', 'OzoneP', 'PM2_5', 'PM2_5_P', 'DieselPM',
       'DieselPM_P', 'Pesticide', 'PesticideP', 'Tox_Rel', 'Tox_Rel_P',
       'Traffic', 'TrafficP', 'DrinkWat', 'DrinkWatP', 'Lead', 'Lead_P',
       'Cleanup', 'CleanupP', 'GWThreat', 'GWThreatP', 'HazWaste', 'HazWasteP',
       'ImpWatBod', 'ImpWatBodP', 'SolWaste', 'SolWasteP', 'PollBurd',
       'PolBurdSc', 'PolBurdP', 'Asthma', 'AsthmaP', 'LowBirtWt', 'LowBirWP',
       'Cardiovas', 'CardiovasP', 'Educatn', 'EducatP', 'Ling_Isol',
       'Ling_IsolP', 'Poverty', 'PovertyP', 'Unempl', 'UnemplP', 'HousBurd',
       'HousBurdP', 'PopChar', 'PopCharSc', 'PopCharP', 'Child_10',
       'Pop_10_64', 'Elderly65', 'Hispanic', 'White', 'AfricanAm', 'NativeAm',
       'OtherMult', 'Shape_Leng', 'Shape_Area', 'AAPI', 'geometry'],
      dtype='object')

In [25]:
pctl_cols = ['CIscoreP', 'OzoneP', 'PM2_5_P', 'DieselPM_P', 'PesticideP', 'Tox_Rel_P',
    'TrafficP', 'DrinkWatP', 'Lead_P', 'CleanupP', 'GWThreatP', 'HazWasteP',
    'ImpWatBodP', 'SolWasteP', 'PolBurdP', 'AsthmaP', 'LowBirWP',
    'CardiovasP', 'EducatP', 'Ling_IsolP', 'PovertyP', 'UnemplP',
    'HousBurdP', 'PopCharP']

race_cols = ['Hispanic', 'White', 'AfricanAm', 'NativeAm', 'OtherMult', 'AAPI']
age_cols = ['Child_10','Pop_10_64', 'Elderly65']
df = shape[pctl_cols + race_cols + age_cols]

In [26]:
df['sum'] = df.loc[:, 'Hispanic'] + df.loc[:, 'White'] + df.loc[:, 'AfricanAm'] + df.loc[:, 'NativeAm'] + df.loc[:, 'OtherMult']+df.loc[:, 'AAPI']
df[race_cols + ['sum']].describe()

/var/folders/0k/jn4h9ptx7_500msv80bg_rcm0000gn/T/ipykernel_54503/1370138027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sum'] = df.loc[:, 'Hispanic'] + df.loc[:, 'White'] + df.loc[:, 'AfricanAm'] + df.loc[:, 'NativeAm'] + df.loc[:, 'OtherMult']+df.loc[:, 'AAPI']


,Hispanic,White,AfricanAm,NativeAm,OtherMult,AAPI,sum
count,8011.000000,8011.000000,8011.000000,8011.000000,8011.000000,8011.000000,8011.000000
mean,38.090437,38.671210,5.566892,0.418729,3.267521,13.985211,100.000000
std,26.434934,25.957581,8.575135,2.225242,2.445139,15.655214,0.000069
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,99.999800
25%,15.671250,14.726950,0.835100,0.000000,1.393800,3.234700,100.000000
50%,30.939400,37.246200,2.565900,0.000000,2.846400,8.429500,100.000000
75%,58.169800,60.991000,6.648450,0.360400,4.703750,18.505750,100.000000
max,100.000000,100.000000,84.708200,100.000000,17.065300,94.549900,100.000200


In [ ]:
def select_high_correlated_vars(df, race_cols, threshold = 0.6):
    corr = df.corr()
    vars = pd.DataFrame()
    for race_col in race_cols:
        race_corr = corr[race_col].drop(race_cols)
        selected = race_corr[abs(race_corr) > threshold].sort_values(ascending=False)
        vars = pd.concat([vars, selected], axis=1)
    return vars

select_high_correlated_vars(df, race_cols)

,Hispanic,White,AfricanAm,NativeAm,OtherMult,AAPI
EducatP,0.853965,-0.775303,NaN,NaN,NaN,NaN
CIscoreP,0.741219,-0.725284,NaN,NaN,NaN,NaN
PopCharP,0.729622,-0.713956,NaN,NaN,NaN,NaN
PovertyP,0.675304,NaN,NaN,NaN,NaN,NaN
Ling_IsolP,0.607499,-0.753909,NaN,NaN,NaN,NaN


CES 4.0 Score: CalEnviroScreen Score, Pollution Score multiplied by Population
Characteristics Score \
Education: Percent of population over 25 with less than a high school education \
Pop. Char. Score: Population Characteristics variable scaled with a range of 0-10. (Used
to calculate CES 4.0 Score) \
Poverty: Percent of population living below two times the federal poverty level \
Linguistic Isolation: Percent limited English speaking households

In [82]:
select_high_correlated_vars(numeric_shape, 'Hispanic')

Educatn       0.868749
EducatP       0.853965
CIscore       0.742291
CIscoreP      0.741219
PopCharP      0.729622
PopChar       0.725642
PopCharSc     0.725642
PovertyP      0.675304
Poverty       0.666989
Ling_IsolP    0.607499
Name: Hispanic, dtype: float64

In [83]:
select_high_correlated_vars(numeric_shape, 'White')

Ling_Isol    -0.643171
CIscore      -0.709671
PopCharSc    -0.712490
PopChar      -0.712490
PopCharP     -0.713956
CIscoreP     -0.725284
Educatn      -0.746480
Ling_IsolP   -0.753909
EducatP      -0.775303
Name: White, dtype: float64